# San Francisco Rental Prices Dashboard

In this notebook, you will compile the visualizations from the previous analysis into functions that can be used for a Panel dashboard.

In [10]:
# initial imports
import os
import pandas as pd
import matplotlib.pyplot as plt
import panel as pn
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from dotenv import load_dotenv
pn.extension('plotly')

# Initialize the Panel Extensions (for Plotly)

In [11]:
# Read the Mapbox API key
load_dotenv("./.env")
mapbox_token = os.getenv("MAPBOX")

# Import Data

In [12]:
# Import the CSVs to Pandas DataFrames
file_path = Path("Data/sfo_neighborhoods_census_data.csv")
sfo_data = pd.read_csv(file_path, index_col="year")

sfo_avg_neighborhood_cost_data = sfo_data.groupby("neighborhood").mean()
sfo_avg_neighborhood_cost_data.sort_values("sale_price_sqr_foot", ascending=False, inplace=True)
sfo_avg_neighborhood_cost_data.reset_index(inplace=True)
grouped_sfo_data = sfo_data.groupby(["year", "neighborhood"]).mean()

coordinate_file_path = Path("Data/neighborhoods_coordinates.csv")
df_neighborhood_locations = pd.read_csv(coordinate_file_path)

new_sfo_merge = sfo_avg_neighborhood_cost_data.merge(df_neighborhood_locations, left_on="neighborhood", right_on="Neighborhood")
new_sfo_merge.drop("Neighborhood", axis=1)

,neighborhood,sale_price_sqr_foot,housing_units,gross_rent,Lat,Lon
0,Union Square District,903.993258,377427.50,2555.166667,37.79101,-122.40210
1,Merced Heights,788.844818,380348.00,3414.000000,37.71993,-122.46595
2,Miraloma Park,779.810842,375967.25,2155.250000,37.73415,-122.45700
3,Pacific Heights,689.555817,378401.00,2817.285714,37.79298,-122.43579
4,Westwood Park,687.087575,382295.00,3959.000000,37.73415,-122.45700
...,...,...,...,...,...,...
64,Visitacion Valley,301.466180,381321.50,3657.000000,37.72874,-122.41098
65,Outer Mission,242.370952,379374.50,2995.750000,37.72280,-122.43869
66,Bayview,204.588623,376454.00,2318.400000,37.73467,-122.40106
67,Hunters Point,170.624920,377427.50,2489.000000,37.72551,-122.37178


- - -

## Panel Visualizations

In this section, you will copy the code for each plot type from your analysis notebook and place it into separate functions that Panel can use to create panes for the dashboard. 

These functions will convert the plot object to a Panel pane.

Be sure to include any DataFrame transformation/manipulation code required along with the plotting code.

Return a Panel pane object from each function that can be used to build the dashboard.

Note: Remove any `.show()` lines from the code. We want to return the plots instead of showing them. The Panel dashboard will then display the plots.

In [13]:
# Define Panel Visualization Functions
def housing_units_per_year():
    """Housing Units Per Year."""
    
    sfo_data_ymin = sfo_data["housing_units"].min() - 1000
    sfo_data_ymax = sfo_data["housing_units"].max() + 1000

    sfo_data.groupby(["year"]).mean().plot(
        kind="bar",
        y="housing_units",
        label="Housing Units",
        ylim=(sfo_data_ymin, sfo_data_ymax)
        )

def average_gross_rent():
    """Average Gross Rent in San Francisco Per Year."""
    
    sfo_data.groupby(["year"]).mean().plot(
        kind="line",
        y="gross_rent",
        label="Average Gross Rent in San Fransisco")

def average_sales_price():
    """Average Sales Price Per Year."""
    
    sfo_data.groupby(["year"]).mean().plot(
        kind="line",
        y="sale_price_sqr_foot",
        label="Sale Price Per Sq Ft"
        )

def average_price_by_neighborhood():
    """Average Prices by Neighborhood."""
    
    grouped_sfo_data.hvplot.line(
        x="year",
        y="sale_price_sqr_foot",
        xlabel="Year",
        ylabel="Sale Price Per Sqr Foot",
        groupby="neighborhood"
        )

def top_most_expensive_neighborhoods():
    """Top 10 Most Expensive Neighborhoods."""
    
    sfo_avg_neighborhood_cost_data.head(10).plot(
        kind="bar",
        y="sale_price_sqr_foot",
        xlabel="SFO Neighborhood",
        ylabel="Sale Price Per Square Foot",
        ylim=(600,950)
        )


def parallel_coordinates():
    """Parallel Coordinates Plot."""
    
    fig = px.parallel_coordinates(
        sfo_avg_neighborhood_cost_data,
        color="sale_price_sqr_foot",
        color_continuous_scale=px.colors.sequential.Inferno,
        labels={"index":"Neighborhood", "sale_price_sqr_foot":"Price Per Square Foot", "housing_units":"Number of Housing Units", "gross_rent":"Gross Rent"})


def parallel_categories():
    """Parallel Categories Plot."""
    
    fig = px.parallel_categories(
        sfo_avg_neighborhood_cost_data.head(10),
        dimensions=["neighborhood", "sale_price_sqr_foot", "housing_units", "gross_rent"],
        color_continuous_scale=px.colors.sequential.Viridis,
        color="sale_price_sqr_foot",
        labels={
            "neighborhood":"Neighborhood",
            "sale_price_sqr_foot":"Price Per Square Foot",
            "housing_units":"Number of Housing Units",
            "gross_rent":"Gross Rent"}
            )

def neighborhood_map():
    """Neighborhood Map"""

    sfo_map = px.scatter_mapbox(
        new_sfo_merge,
        lat="Lat",
        lon="Lon",
        size="sale_price_sqr_foot",
        color="neighborhood",
        zoom=10,
        mapbox_style="open-street-map"
        )


## Panel Dashboard

In this section, you will combine all of the plots into a single dashboard view using Panel. Be creative with your dashboard design!

In [14]:
row_metrics = pn.Row(
    ("Housing Units Per Year", housing_units_per_year),
    ("Average Gross Rent Citywide", average_gross_rent),
    ("Average Sales Price Citywide", average_sales_price)
    )

row_cost = pn.Row(
    ("Average Price by Neighborhood", average_price_by_neighborhood),
    ("Top Most Expensive Neighborhoods", top_most_expensive_neighborhoods),
)
column_compare = pn.Column(
    parallel_categories,
    parallel_coordinates
)

tabular = pn.Tabs(
    row_metrics,
    row_cost,
    column_compare,
    neighborhood_map
)

In [15]:
tabular.servable()

Tabs
    [0] Row
        [0] Str(tuple)
        [1] Str(tuple)
        [2] Str(tuple)
    [1] Row
        [0] Str(tuple)
        [1] Str(tuple)
    [2] Column
        [0] Column
            [0] Column()
            [1] Row
                [0] Str(None, name='interactive01496')
        [1] Column
            [0] Column()
            [1] Row
                [0] Str(None, name='interactive01502')
    [3] Column
        [0] Column()
        [1] Row
            [0] Str(None, name='interactive01509')